In [1]:
from newsapi import NewsApiClient
import csv
import pandas as pd
from textblob import TextBlob
from dateutil import parser
api=NewsApiClient(api_key='5474adf03fac4b10ad2b144b7fdf8732')

In [10]:
all_articles = api.get_everything(q='marijuana',
                                      sources='the-globe-and-mail,financial-post',
                                      language='en')

In [9]:
all_articles

{'status': 'ok',
 'totalResults': 906,
 'articles': [{'source': {'id': 'the-globe-and-mail',
    'name': 'The Globe And Mail'},
   'author': 'Kristine Owram',
   'title': 'First actively run pot ETF starts trading as sector soars',
   'description': 'Redwood Marijuana Opportunities Fund launches in Toronto',
   'url': 'https://www.theglobeandmail.com/globe-investor/funds-and-etfs/etfs/first-actively-run-pot-etf-starts-trading-as-sector-soars/article37814813/',
   'urlToImage': 'https://www.theglobeandmail.com/resizer/F4t_1AEH9qtOfLkTaByPywD09aQ=/1200x0/filters:quality(80)/arc-anglerfish-tgam-prod-tgam.s3.amazonaws.com/public/IBSRN5WP65HZZDCE5YTJOI6Q74.JPG',
   'publishedAt': '2018-02-01T15:04:23Z'},
  {'source': {'id': 'the-globe-and-mail', 'name': 'The Globe And Mail'},
   'author': 'Chris Hannay, Mayaz Alam and James Keller',
   'title': 'Politics Briefing: Alberta, B.C. spat goes from oil to wine',
   'description': 'Also: Marijuana legalization could be delayed',
   'url': 'https:/

In [14]:
news_data=pd.DataFrame(all_articles['articles'])

In [16]:
#Parsing out date and formatting into news_data dataframe
news_data['date']=news_data.apply(lambda x: parser.parse(x['publishedAt']).strftime('%Y-%m-%d'), axis=1)
#Calculating polarity and subjectivity of articles using TextBlob Natural Language Processing (NLP)
news_data['polarity']=news_data.apply(lambda x: TextBlob(x['description']).sentiment.polarity, axis=1)
news_data['subjectivity']=news_data.apply(lambda x: TextBlob(x['description']).sentiment.subjectivity, axis=1)


In [17]:
news_data.head()

,author,description,publishedAt,source,title,url,urlToImage,date,polarity,subjectivity
0,"Chris Hannay, Mayaz Alam and James Keller",Also: Marijuana legalization could be delayed,2018-02-07T13:42:12Z,"{'id': 'the-globe-and-mail', 'name': 'The Glob...","Politics Briefing: Alberta, B.C. spat goes fro...",https://www.theglobeandmail.com/news/politics/...,https://www.theglobeandmail.com/resizer/ORE-uv...,2018-02-07,0.000000,0.000000
1,Kristine Owram,Redwood Marijuana Opportunities Fund launches ...,2018-02-01T15:04:23Z,"{'id': 'the-globe-and-mail', 'name': 'The Glob...",First actively run pot ETF starts trading as s...,https://www.theglobeandmail.com/globe-investor...,https://www.theglobeandmail.com/resizer/F4t_1A...,2018-02-01,0.000000,0.000000
2,None,TSX dips slightly despite marijuana stock rally,2018-03-23T15:29:11Z,"{'id': 'the-globe-and-mail', 'name': 'The Glob...",At midday: Wall Street flat as trade war fears...,https://www.theglobeandmail.com/globe-investor...,https://www.theglobeandmail.com/resizer/ITaiYq...,2018-03-23,-0.166667,0.166667
3,Scott Barlow,A roundup of what The Globe and Mail’s market ...,2018-01-10T14:24:24Z,"{'id': 'the-globe-and-mail', 'name': 'The Glob...",Marijuana stock investors ‘delusional’,https://www.theglobeandmail.com/globe-investor...,https://www.theglobeandmail.com/resizer/ExCnR0...,2018-01-10,0.000000,0.000000
4,None,PLUS: TMX review throws marijuana industry int...,2017-10-20T20:10:07Z,"{'id': 'the-globe-and-mail', 'name': 'The Glob...",Streetwise newsletter: The week’s best reads: ...,https://www.theglobeandmail.com/report-on-busi...,https://beta.theglobeandmail.com/resizer/Uh1mt...,2017-10-20,0.000000,0.000000


In [18]:
#Grouping by mean values of dates
news_scores= news_data.groupby(by= 'date').mean()
news_scores

,polarity,subjectivity
date,,
2017-10-04,-0.250000,0.483333
2017-10-05,0.000000,0.000000
2017-10-09,0.000000,0.000000
2017-10-12,0.000000,1.000000
2017-10-20,0.100000,0.100000
2017-11-01,0.000000,0.000000
2017-11-08,0.000000,0.400000
2017-11-13,0.200000,0.150000
2017-11-14,0.500000,0.500000


In [20]:
#Filtering for dates
news_scores_filtered=news_scores.loc['2018-01-01':'2018-06-23']
#Writing to csv file
news_scores_filtered.to_csv('newsscores.csv',sep=',')